Although in the end this model was not used, our group felt it'd still be appropriate to add to our github for the completeness of our submission to TFWorld competition and to allow the community to use this model if the need arose.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_datasets as tfds
import os
from tensorflow import keras 
from tensorflow.keras.layers import Input, Lambda, Dense, dot, Reshape, Embedding
# from keras.layers.embeddings import Embedding
from tensorflow.keras import backend as K
from keras.preprocessing.sequence import pad_sequences
import numpy as np


A note for the following cell. Althouhg our model is the continuous bag of words version of Word2Vec we used Keras's skipgrams preprocessing for nengative sampling due to how our data wasn't formated as sequences. Read more on these functions [here](https://keras.io/preprocessing/sequence/)

In [0]:
from keras.preprocessing.sequence import skipgrams, make_sampling_table
def sampling(inCitation:list, outCitation:list, window_size=3):
    ''' InCitation the original paper, OutCitation: the papers cited in the original paper'''
    global vocab_size
    vocab = list(set(inCitation))
    for out in outCitation:
      for paper in out:
        vocab.append(paper)
    vocab = list(set(vocab))
    vocab_size = len(vocab)

    sampling_table = make_sampling_table(vocab_size)
    labels = []
    data = []
    target = []
    for i in range(len(inCitation)):
        out = outCitation[i] 
        contexes, label = skipgrams(out, vocab_size, window_size=window_size)
        data.append(contexes)
        labels.append(label)
        target.append([inCitation[i]] * len(label))
    
    return target, data, labels


In [0]:
target, data, labels = sampling([1,2,3,5,6], [[3,6,4], [4,2,1], [1,2,5], [2,5,3], [1,2,3], [5]]) #Dummy data

In [0]:
class Word2CBOW(keras.Model): #I should stop naming things
  def __init__(self, window_size=3, **kwargs):
    super().__init__(**kwargs) #handles standard args (e.g., name)
    #super() is to use the keras.Model class
    #To add: Argument for window size and arg for
    self.embedding_layer = Embedding(7, 768, input_length=2) #Only working with a vocab of 6 to test
    self.window_size = 3
    self.context_window = self.window_size * 2
    self.outvec = Dense(1, activation='sigmoid')
    self.similarity = 0
    self.cbow = Lambda(lambda x: K.mean(x, axis=[-1]))
    self.batched_dot = Lambda(self.bdotFunction)

  def bdotFunction(self, x):
        first = x[0]
        second = x[1]
        return K.batch_dot(first, second, axes=-1)

  def call(self, inputs):
    target_input, context_inputs = inputs

    target_input = keras.layers.InputLayer(input_shape=[1,])(target_input)
    context_inputs = keras.layers.InputLayer(input_shape=[2,])(context_inputs)
    
    target1 = self.embedding_layer(target_input)
    context = self.embedding_layer(context_inputs)
   
    context = self.cbow(context) #Averaging the context vectors
    

    dotted = self.batched_dot([tf.squeeze(target1, axis=0), tf.squeeze(context, axis=0)])

    binary_output = self.outvec(dotted)
    binary_output = tf.squeeze(binary_output)

    return binary_output
model = Word2CBOW()
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
# labels = np.array(np.transpose(labels))
model.fit((np.array(target, dtype=np.int32), np.array(data, dtype=np.int32)), y=np.array(labels), epochs=100, batch_size=1)
# model((np.array(target[0], dtype=np.int32), np.array(data[0], dtype=np.int32)))
#Might have to try train_on_batch

(12,)
(12,)
Train on 5 samples
Epoch 1/100
(12,)
(12,)
(12,)
(12,)
5/5 [==============================] - 1s 113ms/sample - loss: 0.6924
Epoch 2/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6896
Epoch 3/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6876
Epoch 4/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6856
Epoch 5/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6835
Epoch 6/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.6815
Epoch 7/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6793
Epoch 8/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.6771
Epoch 9/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6747
Epoch 10/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6723
Epoch 11/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.6699
Epoch 12/100
5/5 [==============================] - 0s 3ms/sa

In [0]:
np.array(np.transpose(labels)).shape

(12, 5)

In [0]:
np.array(labels).shape

(5, 12)

In [0]:
# target = [1,2,3,5,6]
inputs = (np.array(target[0], dtype=np.int32), np.array(data[0], dtype=np.int32))

In [0]:
target_input, context_inputs = inputs
target_input = keras.layers.InputLayer(input_shape=[1,])(target_input)
context_inputs = keras.layers.InputLayer(input_shape=[2,])(context_inputs)
context_inputs.shape, target_input.shape

(TensorShape([12, 2]), TensorShape([12]))

In [0]:
embedding_layer = Embedding(7, 768, input_length=2)
target = embedding_layer(target_input)
context = embedding_layer(context_inputs)
context.shape, target.shape

(TensorShape([12, 2, 768]), TensorShape([12, 768]))

In [0]:
cbow = Lambda(lambda x: K.mean(x, axis=[1]))
context = cbow(context)
context.shape

TensorShape([12, 768])

In [0]:
dotted = dot([target, context], axes=1)
dotted.shape

TensorShape([12, 1])

In [0]:
outvec = Dense(1, activation='sigmoid')
done = outvec(dotted)
# done.shape
tf.convert_to_tensor(labels[0]).shape, tf.squeeze(done).shape

(TensorShape([12]), TensorShape([12]))